In [9]:
# import the packages
import numpy as np
from scipy.fftpack import dct
from numpy.linalg import lstsq

In [10]:
# -----------------------------------------------------------
# Step 1: Preprocessing (normalize to [-1,1])
# -----------------------------------------------------------
def normalize_signal(x):
    xmax = np.max(x[x > 0]) if np.any(x > 0) else 1
    xmin = np.min(x[x < 0]) if np.any(x < 0) else -1
    x_norm = np.zeros_like(x, dtype=float)

    x_norm[x > 0] = x[x > 0] / xmax
    x_norm[x < 0] = x[x < 0] / abs(xmin)
    return x_norm

In [11]:
# -----------------------------------------------------------
# Step 2: Sparse Linear Prediction (simplified AR coeffs)
# -----------------------------------------------------------
def slp_coefficients(x, K=20):
    N = len(x)
    X = np.zeros((N-K, K))
    for i in range(K, N):
        X[i-K] = x[i-K:i][::-1]
    y = x[K:]
    
    # Least squares (L2) for simplicity, can be extended with L1 sparse solver
    a, _, _, _ = lstsq(X, y, rcond=None)
    return a

In [12]:
# -----------------------------------------------------------
# Step 3: SLP to RC (Levinson recursion simplified)
# -----------------------------------------------------------
def slp_to_rc(a):
    N = len(a)
    rc = np.zeros(N)
    a_curr = a.copy()
    for n in range(N, 0, -1):
        rc[n-1] = -a_curr[-1]
        if n > 1:
            a_prev = a_curr[:-1]
            a_curr = (a_prev + rc[n-1] * a_prev[::-1]) / (1 - rc[n-1]**2)
    return rc

In [13]:
# -----------------------------------------------------------
# Step 4: RC Quantization
# -----------------------------------------------------------
def quantize_rc(rc, delta=0.03):
    rc_q = np.round(rc / delta)
    rc_deq = rc_q * delta
    return rc_q, rc_deq

In [14]:
# -----------------------------------------------------------
# Step 5: RC to SLP Transformation (Levinson recursion)
# -----------------------------------------------------------
def rc_to_slp(rc):
    N = len(rc)
    slp = np.zeros((N, N))
    for m in range(N):
        slp[m, m] = 1
    a = np.zeros(N)
    for k in range(N):
        a[k] = rc[k]
        if k > 0:
            a[:k] += rc[k] * slp[k-1, :k][::-1]
        slp[k, :k+1] = a[:k+1]
    return a

In [15]:
# -----------------------------------------------------------
# Step 6: Residual Computation
# -----------------------------------------------------------
def compute_residual(x, a):
    K = len(a)
    N = len(x)
    r = np.zeros(N)
    for n in range(K, N):
        r[n] = x[n] + np.sum(a * x[n-K:n][::-1])
    return r

In [16]:
# -----------------------------------------------------------
# Step 7: Residual DCT + Perceptual Quantization
# -----------------------------------------------------------
def residual_dct_quant(r, asf, delta=0.1):
    R = dct(r, type=2, norm='ortho')
    eq = np.round(R / (delta * asf))
    eq_deq = eq * delta * asf
    return R, eq, eq_deq

In [17]:
# -----------------------------------------------------------
# Example Run
# -----------------------------------------------------------
if __name__ == "__main__":
    # Fake vibrotactile input
    np.random.seed(0)
    x = np.random.randn(200)  # one frame

    # Step 1: Normalize
    x_norm = normalize_signal(x)

    # Step 2: SLP coefficients
    a = slp_coefficients(x_norm, K=20)

    # Step 3: RC
    rc = slp_to_rc(a)

    # Step 4: Quantize RCs
    rc_q, rc_deq = quantize_rc(rc)

    # Step 5 (corrected): Convert dequantized RC back to SLP
    a_deq = rc_to_slp(rc_deq)

    # Step 6: Residual
    r = compute_residual(x_norm, a_deq)

    # Step 7: Residual DCT + Perceptual Quantization
    asf = np.ones_like(r)  # placeholder ASF = 1
    R, eq, eq_deq = residual_dct_quant(r, asf)

    # Print key variables
    print("Normalized x:", x_norm[:10])
    print("SLP coeffs:", a[:5])
    print("RCs:", rc[:5])
    print("Quantized RCs:", rc_q[:5])
    print("Residual:", r[:10])
    print("DCT coeffs:", R[:10])
    print("Quantized DCT:", eq[:10])

Normalized x: [ 0.74022039  0.16791141  0.41069179  0.9403093   0.78365276 -0.38279741
  0.39867004 -0.05928626 -0.04043058  0.17229272]
SLP coeffs: [ 0.01655342  0.09295412  0.06908436 -0.12985137  0.03706271]
RCs: [-0.01422168 -0.17952929 -0.1452085   0.16868508  0.01185   ]
Quantized RCs: [-0. -6. -5.  6.  0.]
Residual: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
DCT coeffs: [ 0.13749397 -0.18743843 -0.45977314  0.26512608  0.70439719 -0.08853236
 -0.13315783 -0.40771887  0.25241097 -0.38712995]
Quantized DCT: [ 1. -2. -5.  3.  7. -1. -1. -4.  3. -4.]
